# Analisi dei corrispettivi dell'azienda <<Là di Cjastelan>>

## Intro

### Descrizione dei dati grezzi

I dati vanno da gennaio 2018 a maggio 2025.

Dal 2018 a maggio 2022 i dati sono stati raccolti manualmente da Riccardo Castellani, da giugno 2022 a marzo 2024 la raccolta, sempre manuale, è stata fatta da Dario Castellani, mentre da aprile 2024 a oggi viene utilizzato un Google Form per far inserire direttamente agli operatori di cassa i vari corrispettivi della giornata.

La raccolta manuale è stata fatta su diversi fogli elettronici Excel. I file prodotti da Riccardo (2018-2022) raccolgono i corrispettivi di un intero anno solare e sono composti da 12 fogli elettronici, uno per ciascun mese dell'anno. I file prodotti da Dario sono annuali per il 2022 e il 2023, divisi per trimestri per il 2024; manca il file relativo all'ultimo trimestre del 2024, essendo questo stato compilato direttamente su cloud in un GoogleDoc. L'ultimo file Excel prodotto da Dario riguarda il primo trimestre del 2025.

Il file del 2022 esiste in doppia versione, una prodotta da Riccardo e una da Dario. La versione di Riccardo verrà ignorata nell'analisi: si tratta, infatti, di una collezione parziale, mancando dell'ultimo trimestre e avendo registrazioni non complete nel trimestre estivo; il file di Dario, invece, duplica tutti i dati corretti della versione di Riccardo e completa e corregge le informazioni mancanti, oltre ad avere i dati dell'ultimo trimestre.

La struttura interna dei fogli di calcolo è simile per ciascun foglio, al netto di aggiunte di casse e reparti che sono arrivate negli anni. I dati sono organizzati in forma tabulare, divisi sulle righe per giorno e sulle colonne per reparti di IVA, raggruppati per cassa. Ai margini ha i totali complessivi e per reparto.    
Nel 2018, la cassa 1 ha 3 reparti: `IVA 4`, `IVA 10` e `IVA 22`, in `IVA 10` collassano i corrispettivi dei reparti 2 e 3. La cassa 2 ha, invece, 4 reparti: `IVA 4`, due `IVA 10` con lo stesso nome ma diversi, probabilmente il primo associato  al Reparto 3 e il secondo al Reparto 2, e un `IVA 22`.

---

## Importazione dei dati

Cominciamo selezionando solo i file che ci interessano, tra tutti quelli consegnati.
Di seguito la struttura della directory `data/raw` dove ho salvato, ed estratto, il file `corrispettivi.zip` contenente tutti i dati consegnati dall'azienda.

In [1]:
import os

DATA_DIR = os.path.join(os.getcwd(), 'data')
RAW_DIR = os.path.join(DATA_DIR, 'raw')

Per motivi di privacy, i dati non sono presenti nel repository. Per questo motivo, riportiamo una rappresentazione testuale della struttura della directory `data/raw`, in modo da poter seguire più agevolmente il processo di selezione dei file utili.

In [2]:
!tree -a {RAW_DIR}

/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Progetto/data/raw
├── corrispettivi.zip
├── Dario
│   ├── CORRISPETTIVI_2022.xlsx
│   ├── CORRISPETTIVI_2023.xlsx
│   ├── corrispettivi 2024.xlsx
│   ├── Corrispettivi apr-mag-giu 2024.xlsx
│   ├── corrispettivi gennaio marzo 2025.xlsx
│   ├── corrispettivi lug-ago-set 2024.xlsx
│   └── .~lock.corrispettivi gennaio marzo 2025.xlsx#
└── Riccardo
    ├── 2019
    │   ├── corrispettivi_ago_19.pdf
    │   ├── corrispettivi_apr_19.pdf
    │   ├── corrispettivi_dic_19.pdf
    │   ├── corrispettivi_feb_19.pdf
    │   ├── corrispettivi_gen_19.pdf
    │   ├── corrispettivi_giu_19.pdf
    │   ├── corrispettivi_lug_19.pdf
    │   ├── corrispettivi_mag_19.pdf
    │   ├── corrispettivi_mar_19.pdf
    │   ├── corrispettivi_nov_19.pdf
    │   ├── corrispettivi_ott_19.pdf
    │   └── corrispettivi_set_19.pdf
    ├── 2020
    │   ├── corrispettivi_ago_2020.pdf
    │   ├── corrispettivi_apr_20.pdf
    │   ├── corrispettivi_dic_2020.pd

I file sono divisi in due principali subdirectory nominate in base a chi ha compilato le tabelle.

Nella subdirectory `Riccardo` sono presenti anche file `.pdf` che altro non sono che le esportazioni dei singoli fogli elettronici nei file Excel, inviati poi al commercialista. I file `288_2.pdf` e `griglia_acq.pdf` offrono una sintesi di altri dati commerciali e di fatturazione, che però non riguardano i corrispettivi ma altri atti di compravendita: probabilmente sono stati inseriti per errore. Il file `CORRISPETTIVI.xlsx` invece è un modello vuoto usato come base per la compilazione di tutti gli altri file

In sintesi, dunque, dobbiamo selezionare solo i file Excel (tutti terminanti in `.xlsx`), avendo cura di scartare i file `Riccardo/CORRISPETTIVI_2022.xlsx` e `Riccardo/CORRISPETTIVI.xlsx` che non sono informativi e i file `.~lock` di sistema.

In [5]:
with os.scandir(RAW_DIR) as rd:
    data_files = []

    for fname in rd:
        if fname.is_dir():
            if fname.name == 'Dario':
                with os.scandir(fname) as dario_sub_dir:
                    data_files += [f.path for f in dario_sub_dir if not f.name.startswith('.')]
            else: # siamo in `/Riccardo`
                with os.scandir(fname) as riccardo_sub_dir:
                    data_files += [
                        f.path for f in riccardo_sub_dir
                            if f.is_file()                  # escludo le altre subdirectory
                            and '.xlsx' in f.name           # escludo i .pdf
                            and '_' in f.name               # escludo il modello vuoto
                            and not f.name.startswith('.')  # escludo i file di sistema
                            and not '2022' in f.name        # escludo il file parziale
                    ] 

for f in data_files:
    print(f)

len(data_files)

/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Progetto/data/raw/Dario/corrispettivi 2024.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Progetto/data/raw/Dario/Corrispettivi apr-mag-giu 2024.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Progetto/data/raw/Dario/corrispettivi gennaio marzo 2025.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Progetto/data/raw/Dario/corrispettivi lug-ago-set 2024.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Progetto/data/raw/Dario/CORRISPETTIVI_2022.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Progetto/data/raw/Dario/CORRISPETTIVI_2023.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Progetto/data/raw/Riccardo/CORRISPETTIVI_2018.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Progetto/data/raw/Riccardo/CORRISPETTIVI_2019.xlsx
/media/yuuki

10

Dei 55 file consegnati, abbiamo ora selezionato e salvato in `data_files` i 10 file utili che dovremo importare in Pandas.

Per requisito, la forma del DataFrame finale deve rispecchiare quella del file più recente (del 2025). Usiamo allora un'indicizzazione gerarchica sulle colonne, indicizzando per cassa (dalla 1 alla 3) e specificando, per ogni cassa, gli incassi di ciascun reparto dall'1 al 4. Ci sarà infine un gruppo marginale `Totali`, diviso a sua volta in `IVA 4%`, `IVA 10%`, `IVA 10% comm.` e il totale giornaliero.

La funzione `import_data`, a cui è delegata l'importazione dei file Excel, è molto complessa: ciò dipende dal fatto che i file, e i fogli all'interno di essi, non hanno tutti esattamente la stessa struttura, ma presentano lievi irregolarità sufficienti a rendere impossibile scrivere una procedura abbastanza generale da essere chiamata su ogni file. Per questo motivo, di seguito ci si soffermerà sugli aspetti più salienti della procedura, delegando al codice e ai commenti al codice gli aspetti più puntuali.

Svolgiamo l'importazione usando il metodo `pd.read_excel()`. In questo modo possiamo sfruttare i dati direttamente nel formato con cui ci sono stati passati, senza doverli manipolare manualmente per convertirli in formati più facili da importare, come per esempio in `.csv`. Questo metodo, inoltre, ci fornisce una serie di funzionalità già pronte per poter gestire e tradurre direttamente le particolarità di come sono stati organizzati i dati nei file Excel, come per esempio la divisione in più fogli elettronici in ciascun file.

Chiamiamo `pd.read_excel()` con queste opzioni:
 - `sheet_name=None` per importare tutti i fogli elettronici di cui ciascun file è composto. Questa opzione fa in modo che il metodo non 
 ritorni un DataFrame ma un dizionario, le cui chiavi sono i nomi dei fogli di cui è composto il file e i valori sono il DataFrame importato;
 - `header=[2,3]` per usare le righe 3 e 4 del file Excel per costruire un `MultiIndex` per le colonne del DataFrame;
 - `skipfooter=1` per non importare l'ultima riga di ogni foglio che contiene i totali del mese.

_Nota_: per poter usare il metodo `pd.read_excel()` potrebbe essere necessario installare, tramite `pip`, la dipendenza opzionale `openpyxl`.

Mentre importiamo i dati, dobbiamo codificare le informazioni sul mese e sull'anno che non sono presenti nei dati, ma rispettivamente nel nome dei fogli elettronici e nel nome del file.

In [88]:
import numpy as np
import pandas as pd

def import_datas(excel_files):
    '''
    Helpers
    '''
    
    def flatten(l):
        '''
        Appiattisce una lista di liste
        '''
        return [el for subl in l for el in subl]
    
    def make_multicols(n_casse):
        '''
        Crea modularmente un MultiIndex della forma:
        Data       Giorno, Mese, Anno, 
        Cassa n    Reparto 1 ... Reparto 5 (in base a quante n_casse chiediamo)
        Totali     IVA 4, IVA 10, IVA 10 comm, Tot Giornaliero


        n_casse : int
            il numero di casse registrate
        '''

        level1 = ['Data'] + [f'Cassa {n}' for n in range(1, n_casse + 1)] + ['Totali']
        level2 = ['Giorno', 'Mese', 'Anno', 'Reparto 1', 'Reparto 2', 'Reparto 3', 'Reparto 4', 'Reparto 5', 'IVA 4', 'IVA 10', 'IVA 10 comm', 'Tot Giornaliero']

        level1_casse_codes = flatten([[n] * 5 for n in range(1, n_casse + 1)])
        level2_casse_codes = [3, 4, 5, 6, 7] * n_casse
        level1_codes = [0] * 3 + level1_casse_codes + [n_casse + 1] * 4
        level2_codes = [0, 1, 2] + level2_casse_codes + [8, 9, 10, 11]

        return pd.MultiIndex(levels=[level1, level2], codes=[level1_codes, level2_codes])        

    def tidy_date(sheet, df, fname, tidy):
        '''
        Compila le colonne Data di tidy in base ai dati sporchi del df che si sta importando

        sheet: chiave del dictionary di df ritornato da pd.read_excel()
        df: valore associato a sheet
        fname: nome del file che si sta importando
        '''

        tidy[('Data', 'Giorno')] = df.iloc[:, 0]
        tidy[('Data', 'Mese')] = sheet.lower()
        #df['Mese'] = df['Mese'].astype(pd.StringDtype())
        tidy[('Data', 'Anno')] = int(fname.split('.')[-2][-4:])

    def tidy_casse(df, tidy, anno):
        '''
        Compila le colonne dei corrispettivi delle casse in tidy associando i dati giusti del df che si sta importando da pd.read_excel()
        '''

        iva_rep_map = {
            'IVA 4': 'Reparto 1',        # formaggi e latticini
            'IVA 10_c': 'Reparto 2',     # carne e insaccati
            'IVA 10_y': 'Reparto 3',     # yogurt
            'IVA 10 comm': 'Reparto 4',  # trasformati
            'IVA 22': 'Reparto 5'
        }

        # se i corrispettivi sono segnati sotto 'IVA' invece che per 'Reparto'
        cols_lv2 = df.columns.get_level_values(1)
        if cols_lv2.map(lambda x: 'IVA' in x).any():
            casse = [tup for tup in df.columns if 'Cassa' in tup[0].capitalize()]
            print(casse)

            for cassa in casse:
                if anno == 2018 and 'IVA 10' in cassa[1]:
                    pass
                else:
                    tidy[(cassa[0].capitalize(), iva_rep_map[cassa[1]])] = df[cassa]
        
    '''
    Main Routine
    '''
    
    # creo il DataFrame da restituire come risultato
    res = pd.DataFrame(columns=make_multicols(3))
    
    for f in excel_files:
        if '2018' in f:
            df_dict = pd.read_excel(f, sheet_name=None, header=[2, 3], skipfooter=1)

            # creo un DataFrame ben formattato dove copiare i dati importati da poter poi concatenare al risultato
            tidy = pd.DataFrame(columns=make_multicols(2))
            
            for sheet, df in df_dict.items():
                df.fillna(0, inplace=True)
                display(df)
                
                tidy_date(sheet, df, f, tidy)
                tidy_casse(df, tidy, tidy[('Data', 'Anno')][0])
                
                display(tidy)

import_datas(data_files)

GIORNO CASSA 1                CASSA 2                        \
   Unnamed: 0_level_1   IVA 4  IVA 10 IVA 22   IVA 4 IVA 10 IVA 10  IVA 22   
0                   1    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
1                   2  267.02   57.07    0.0    0.00   0.00    0.00    0.0   
2                   3  161.85   74.85    0.0    0.00   0.00    0.00    0.0   
3                   4  354.17   94.98    0.0  277.32  18.08   15.06    0.0   
4                   5    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
5                   6    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
6                   7    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
7                   8  361.54   71.20    0.0    0.00   0.00    0.00    0.0   
8                   9   71.19   30.38    0.0    0.00   0.00    0.00    0.0   
9                  10  259.57   25.00    0.0    0.00   0.00    0.00    0.0   
10                 11  402.82   37.28    0.0  147.88  15.56    0.00    0.0   
11                 12  349.10   29.40    0.0    0.00   0.00    0.00    0.0   
12                 13  353.83  129.16    0.0  255.63  18.50   27.21    0.0   
13                 14    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
14                 15    0.00    0.00    0.0  231.48   2.00   18.10    0.0   
15                 16   45.44   12.10    0.0    0.00   0.00    0.00    0.0   
16                 17  272.75  118.47    0.0   51.30   0.00  386.94    0.0   
17                 18  344.34  299.17    0.0  307.85   8.00  167.58    0.0   
18                 19  279.67  272.54    0.0    0.00   0.00    0.00    0.0   
19                 20  271.93  163.86    0.0  324.10  21.30  286.27    0.0   
20                 21    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
21                 22  456.63   15.85    0.0    0.00   0.00    0.00    0.0   
22                 23   81.62   66.78    0.0    0.00   0.00    0.00    0.0   
23                 24  406.15  120.89    0.0    0.00   0.00    0.00    0.0   
24                 25  339.48   19.86    0.0  264.00  19.25    0.00    0.0   
25                 26  302.63   10.00    0.0    0.00   0.00    0.00    0.0   
26                 27  368.97   81.23    0.0  253.64  11.25   49.01    0.0   
27                 28    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
28                 29  267.33   14.05    0.0    0.00   0.00    0.00    0.0   
29                 30  185.05    2.75    0.0    0.00   0.00    0.00    0.0   
30                 31  298.76    5.50    0.0    0.00   0.00    0.00    0.0   

   CORRISPETTIVI                          
           IVA 4  IVA 10 IVA 22   TOTALE  
0           0.00    0.00      0     0.00  
1         267.02   57.07      0   324.09  
2         161.85   74.85      0   236.70  
3         631.49  128.12      0   759.61  
4           0.00    0.00      0     0.00  
5           0.00    0.00      0     0.00  
6           0.00    0.00      0     0.00  
7         361.54   71.20      0   432.74  
8          71.19   30.38      0   101.57  
9         259.57   25.00      0   284.57  
10        550.70   52.84      0   603.54  
11        349.10   29.40      0   378.50  
12        609.46  174.87      0   784.33  
13          0.00    0.00      0     0.00  
14        231.48   20.10      0   251.58  
15         45.44   12.10      0    57.54  
16        324.05  505.41      0   829.46  
17        652.19  474.75      0  1126.94  
18        279.67  272.54      0   552.21  
19        596.03  471.43      0  1067.46  
20          0.00    0.00      0     0.00  
21        456.63   15.85      0   472.48  
22         81.62   66.78      0   148.40  
23        406.15  120.89      0   527.04  
24        603.48   39.11      0   642.59  
25        302.63   10.00      0   312.63  
26        622.61  141.49      0   764.10  
27          0.00    0.00      0     0.00  
28        267.33   14.05      0   281.38  
29        185.05    2.75      0   187.80  
30        298.76    5.50      0   304.26

[('CASSA 1', 'IVA 4'), ('CASSA 1', 'IVA 10'), ('CASSA 1', 'IVA 22'), ('CASSA 2', 'IVA 4'), ('CASSA 2', 'IVA 10'), ('CASSA 2', 'IVA 10 '), ('CASSA 2', 'IVA 22')]


Data                  Cassa 1                                          \
   Giorno     Mese  Anno Reparto 1 Reparto 2 Reparto 3 Reparto 4 Reparto 5   
0       1  gennaio  2018      0.00       NaN       NaN       NaN       0.0   
1       2  gennaio  2018    267.02       NaN       NaN       NaN       0.0   
2       3  gennaio  2018    161.85       NaN       NaN       NaN       0.0   
3       4  gennaio  2018    354.17       NaN       NaN       NaN       0.0   
4       5  gennaio  2018      0.00       NaN       NaN       NaN       0.0   
5       6  gennaio  2018      0.00       NaN       NaN       NaN       0.0   
6       7  gennaio  2018      0.00       NaN       NaN       NaN       0.0   
7       8  gennaio  2018    361.54       NaN       NaN       NaN       0.0   
8       9  gennaio  2018     71.19       NaN       NaN       NaN       0.0   
9      10  gennaio  2018    259.57       NaN       NaN       NaN       0.0   
10     11  gennaio  2018    402.82       NaN       NaN       NaN       0.0   
11     12  gennaio  2018    349.10       NaN       NaN       NaN       0.0   
12     13  gennaio  2018    353.83       NaN       NaN       NaN       0.0   
13     14  gennaio  2018      0.00       NaN       NaN       NaN       0.0   
14     15  gennaio  2018      0.00       NaN       NaN       NaN       0.0   
15     16  gennaio  2018     45.44       NaN       NaN       NaN       0.0   
16     17  gennaio  2018    272.75       NaN       NaN       NaN       0.0   
17     18  gennaio  2018    344.34       NaN       NaN       NaN       0.0   
18     19  gennaio  2018    279.67       NaN       NaN       NaN       0.0   
19     20  gennaio  2018    271.93       NaN       NaN       NaN       0.0   
20     21  gennaio  2018      0.00       NaN       NaN       NaN       0.0   
21     22  gennaio  2018    456.63       NaN       NaN       NaN       0.0   
22     23  gennaio  2018     81.62       NaN       NaN       NaN       0.0   
23     24  gennaio  2018    406.15       NaN       NaN       NaN       0.0   
24     25  gennaio  2018    339.48       NaN       NaN       NaN       0.0   
25     26  gennaio  2018    302.63       NaN       NaN       NaN       0.0   
26     27  gennaio  2018    368.97       NaN       NaN       NaN       0.0   
27     28  gennaio  2018      0.00       NaN       NaN       NaN       0.0   
28     29  gennaio  2018    267.33       NaN       NaN       NaN       0.0   
29     30  gennaio  2018    185.05       NaN       NaN       NaN       0.0   
30     31  gennaio  2018    298.76       NaN       NaN       NaN       0.0   

     Cassa 2                                         Totali         \
   Reparto 1 Reparto 2 Reparto 3 Reparto 4 Reparto 5  IVA 4 IVA 10   
0       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
1       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
2       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
3     277.32       NaN       NaN       NaN       0.0    NaN    NaN   
4       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
5       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
6       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
7       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
8       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
9       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
10    147.88       NaN       NaN       NaN       0.0    NaN    NaN   
11      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
12    255.63       NaN       NaN       NaN       0.0    NaN    NaN   
13      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
14    231.48       NaN       NaN       NaN       0.0    NaN    NaN   
15      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
16     51.30       NaN       NaN       NaN       0.0    NaN    NaN   
17    307.85       NaN       NaN       NaN       0.0    NaN    NaN   
18      0.00       NaN       N

GIORNO CASSA 1                CASSA 2                        \
   Unnamed: 0_level_1   IVA 4  IVA 10 IVA 22   IVA 4 IVA 10 IVA 10  IVA 22   
0                   1  193.22   69.86    0.0  186.08   4.75    0.00    0.0   
1                   2  243.42   15.67    0.0    0.00   0.00    0.00    0.0   
2                   3  281.19   82.70    0.0  216.39   5.00    0.00    0.0   
3                   4    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
4                   5  397.75   32.00    0.0    0.00   0.00    0.00    0.0   
5                   6  165.73   17.86    0.0    0.00   0.00    0.00    0.0   
6                   7    0.00    0.00    0.0  237.47   5.46    0.00    0.0   
7                   8  206.16   43.10    0.0  275.86   5.78   23.28    0.0   
8                   9  356.96   20.72    0.0    0.00   0.00    0.00    0.0   
9                  10  322.14   78.10    0.0  300.34  20.41   39.82    0.0   
10                 11    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
11                 12  185.72   22.46    0.0    0.00   0.00    0.00    0.0   
12                 13  180.43    4.85    0.0    0.00   0.00    0.00    0.0   
13                 14   28.50  163.90    0.0  213.78  10.96  122.30    0.0   
14                 15  348.17  261.16    0.0  265.28   4.00  161.08    0.0   
15                 16  326.69  313.95    0.0    0.00   0.00    0.00    0.0   
16                 17  300.22  162.00    0.0  258.68  20.25  367.02    0.0   
17                 18    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
18                 19  393.90   40.13    0.0    0.00   0.00    0.00    0.0   
19                 20  259.89   31.47    0.0    0.00   0.00    0.00    0.0   
20                 21  464.12   88.86    0.0    0.00   0.00    0.00    0.0   
21                 22  316.85   77.69    0.0  317.47   4.75   18.12    0.0   
22                 23  150.62    0.00    0.0    0.00   0.00    0.00    0.0   
23                 24  333.77  105.80    0.0  345.13  16.50    0.00    0.0   
24                 25    0.00    0.00    0.0  382.62   0.00    0.00    0.0   
25                 26  158.18    2.50    0.0    0.00   0.00    0.00    0.0   
26                 27  127.29    4.75    0.0    0.00   0.00    0.00    0.0   
27                 28    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
28                 29    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
29                 30    0.00    0.00    0.0    0.00   0.00    0.00    0.0   
30                 31    0.00    0.00    0.0    0.00   0.00    0.00    0.0   

   CORRISPETTIVI                          
           IVA 4  IVA 10 IVA 22   TOTALE  
0         379.30   74.61    0.0   453.91  
1         243.42   15.67    0.0   259.09  
2         497.58   87.70    0.0   585.28  
3           0.00    0.00    0.0     0.00  
4         397.75   32.00    0.0   429.75  
5         165.73   17.86    0.0   183.59  
6         237.47    5.46    0.0   242.93  
7         482.02   72.16    0.0   554.18  
8         356.96   20.72    0.0   377.68  
9         622.48  138.33    0.0   760.81  
10          0.00    0.00    0.0     0.00  
11        185.72   22.46    0.0   208.18  
12        180.43    4.85    0.0   185.28  
13        242.28  297.16    0.0   539.44  
14        613.45  426.24    0.0  1039.69  
15        326.69  313.95    0.0   640.64  
16        558.90  549.27    0.0  1108.17  
17          0.00    0.00    0.0     0.00  
18        393.90   40.13    0.0   434.03  
19        259.89   31.47    0.0   291.36  
20        464.12   88.86    0.0   552.98  
21        634.32  100.56    0.0   734.88  
22        150.62    0.00    0.0   150.62  
23        678.90  122.30    0.0   801.20  
24        382.62    0.00    0.0   382.62  
25        158.18    2.50    0.0   160.68  
26        127.29    4.75    0.0   132.04  
27          0.00    0.00    0.0     0.00  
28          0.00    0.00    0.0     0.00  
29          0.00    0.00    0.0     0.00  
30          0.00    0.00    0.0     0.00

[('CASSA 1', 'IVA 4'), ('CASSA 1', 'IVA 10'), ('CASSA 1', 'IVA 22'), ('CASSA 2', 'IVA 4'), ('CASSA 2', 'IVA 10'), ('CASSA 2', 'IVA 10 '), ('CASSA 2', 'IVA 22')]


...

[('CASSA 1', 'IVA 4'), ('CASSA 1', 'IVA 10'), ('CASSA 1', 'IVA 22'), ('CASSA 2', 'IVA 4'), ('CASSA 2', 'IVA 10'), ('CASSA 2', 'IVA 10 '), ('CASSA 2', 'IVA 22')]


Data                   Cassa 1                                          \
   Giorno      Mese  Anno Reparto 1 Reparto 2 Reparto 3 Reparto 4 Reparto 5   
0       1  novembre  2018      0.00       NaN       NaN       NaN       0.0   
1       2  novembre  2018    321.89       NaN       NaN       NaN       0.0   
2       3  novembre  2018    253.68       NaN       NaN       NaN       0.0   
3       4  novembre  2018      0.00       NaN       NaN       NaN       0.0   
4       5  novembre  2018    252.42       NaN       NaN       NaN       0.0   
5       6  novembre  2018    129.84       NaN       NaN       NaN       0.0   
6       7  novembre  2018    518.52       NaN       NaN       NaN       0.0   
7       8  novembre  2018    383.86       NaN       NaN       NaN       0.0   
8       9  novembre  2018    431.81       NaN       NaN       NaN       0.0   
9      10  novembre  2018    215.84       NaN       NaN       NaN       0.0   
10     11  novembre  2018      0.00       NaN       NaN       NaN       0.0   
11     12  novembre  2018    327.05       NaN       NaN       NaN       0.0   
12     13  novembre  2018    171.29       NaN       NaN       NaN       0.0   
13     14  novembre  2018    269.25       NaN       NaN       NaN       0.0   
14     15  novembre  2018    280.18       NaN       NaN       NaN       0.0   
15     16  novembre  2018      0.00       NaN       NaN       NaN       0.0   
16     17  novembre  2018    230.75       NaN       NaN       NaN       0.0   
17     18  novembre  2018      0.00       NaN       NaN       NaN       0.0   
18     19  novembre  2018    220.94       NaN       NaN       NaN       0.0   
19     20  novembre  2018     40.97       NaN       NaN       NaN       0.0   
20     21  novembre  2018    491.17       NaN       NaN       NaN       0.0   
21     22  novembre  2018    289.84       NaN       NaN       NaN       0.0   
22     23  novembre  2018    470.09       NaN       NaN       NaN       0.0   
23     24  novembre  2018    212.27       NaN       NaN       NaN       0.0   
24     25  novembre  2018      0.00       NaN       NaN       NaN       0.0   
25     26  novembre  2018    386.00       NaN       NaN       NaN       0.0   
26     27  novembre  2018    144.01       NaN       NaN       NaN       0.0   
27     28  novembre  2018    305.61       NaN       NaN       NaN       0.0   
28     29  novembre  2018    266.48       NaN       NaN       NaN       0.0   
29     30  novembre  2018    210.31       NaN       NaN       NaN       0.0   
30     31  novembre  2018      0.00       NaN       NaN       NaN       0.0   

     Cassa 2                                         Totali         \
   Reparto 1 Reparto 2 Reparto 3 Reparto 4 Reparto 5  IVA 4 IVA 10   
0     292.03       NaN       NaN       NaN       0.0    NaN    NaN   
1       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
2     320.78       NaN       NaN       NaN       0.0    NaN    NaN   
3       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
4       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
5       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
6     156.68       NaN       NaN       NaN       0.0    NaN    NaN   
7     161.31       NaN       NaN       NaN       0.0    NaN    NaN   
8       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
9     305.74       NaN       NaN       NaN       0.0    NaN    NaN   
10      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
11      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
12      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
13      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
14    319.64       NaN       NaN       NaN       0.0    NaN    NaN   
15      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
16    234.46       NaN       NaN       NaN       0.0    NaN    NaN   
17      0.00       NaN       NaN       NaN       0.0    NaN    NaN 

GIORNO CASSA 1                CASSA 2                         \
   Unnamed: 0_level_1   IVA 4  IVA 10 IVA 22   IVA 4  IVA 10 IVA 10  IVA 22   
0                   1  309.38   90.62    0.0    0.00    0.00    0.00    0.0   
1                   2  551.91    0.00    0.0    0.00    0.00    0.00    0.0   
2                   3  287.91    3.89    0.0    0.00    0.00    0.00    0.0   
3                   4  416.46    0.00    0.0    0.00    0.00    0.00    0.0   
4                   5  556.58    0.00    0.0   60.51  213.07    0.00    0.0   
5                   6  320.02   29.69    0.0  196.46    7.50   51.52    0.0   
6                   7  300.20    4.41    0.0    0.00    0.00    0.00    0.0   
7                   8    0.00    0.00    0.0  460.56   15.75  172.37    0.0   
8                   9    0.00    0.00    0.0    0.00    0.00    0.00    0.0   
9                  10  347.69    4.75    0.0    0.00    0.00    0.00    0.0   
10                 11  274.62   10.56    0.0    0.00    0.00    0.00    0.0   
11                 12  361.90  200.15    0.0    0.00    0.00    0.00    0.0   
12                 13  245.48  328.23    0.0  360.43    5.00    0.00    0.0   
13                 14    0.00    0.00    0.0    0.00    0.00    0.00    0.0   
14                 15  856.55    0.00    0.0  530.66   10.50    0.00    0.0   
15                 16    0.00    0.00    0.0    0.00    0.00    0.00    0.0   
16                 17  435.76  135.59    0.0    0.00    0.00    0.00    0.0   
17                 18  212.82   80.17    0.0    0.00    0.00    0.00    0.0   
18                 19  890.08   46.61    0.0    0.00    0.00    0.00    0.0   
19                 20  257.62  113.97    0.0  124.76    0.00    0.00    0.0   
20                 21  503.01    0.00    0.0    0.00    0.00    0.00    0.0   
21                 22  238.30  142.66    0.0  346.69   10.50    0.00    0.0   
22                 23    0.00    0.00    0.0    0.00    0.00    0.00    0.0   
23                 24  466.28   10.00    0.0    0.00    0.00    0.00    0.0   
24                 25    0.00    0.00    0.0    0.00    0.00    0.00    0.0   
25                 26    0.00    0.00    0.0    0.00    0.00    0.00    0.0   
26                 27  185.45    3.39    0.0  158.20    6.00    0.00    0.0   
27                 28  206.09    0.00    0.0    0.00    0.00    0.00    0.0   
28                 29  206.99  125.36    0.0  203.93    7.43    0.00    0.0   
29                 30  386.69   29.73    0.0    0.00    0.00    0.00    0.0   
30                 31    0.00    0.00    0.0    0.00    0.00    0.00    0.0   

   CORRISPETTIVI                          
           IVA 4  IVA 10 IVA 22   TOTALE  
0         309.38   90.62      0   400.00  
1         551.91    0.00      0   551.91  
2         287.91    3.89      0   291.80  
3         416.46    0.00      0   416.46  
4         617.09  213.07      0   830.16  
5         516.48   88.71      0   605.19  
6         300.20    4.41      0   304.61  
7         460.56  188.12      0   648.68  
8           0.00    0.00      0     0.00  
9         347.69    4.75      0   352.44  
10        274.62   10.56      0   285.18  
11        361.90  200.15      0   562.05  
12        605.91  333.23      0   939.14  
13          0.00    0.00      0     0.00  
14       1387.21   10.50      0  1397.71  
15          0.00    0.00      0     0.00  
16        435.76  135.59      0   571.35  
17        212.82   80.17      0   292.99  
18        890.08   46.61      0   936.69  
19        382.38  113.97      0   496.35  
20        503.01    0.00      0   503.01  
21        584.99  153.16      0   738.15  
22          0.00    0.00      0     0.00  
23        466.28   10.00      0   476.28  
24          0.00    0.00      0     0.00  
25          0.00    0.00      0     0.00  
26        343.65    9.39      0   353.04  
27        206.09    0.00      0   206.09  
28        410.92  132.79      0   543.71  
29        386.69   29.73      0   416.42  
30          0.00    0.00      0

[('CASSA 1', 'IVA 4'), ('CASSA 1', 'IVA 10'), ('CASSA 1', 'IVA 22'), ('CASSA 2', 'IVA 4'), ('CASSA 2', 'IVA 10'), ('CASSA 2', 'IVA 10 '), ('CASSA 2', 'IVA 22')]


Data                   Cassa 1                                          \
   Giorno      Mese  Anno Reparto 1 Reparto 2 Reparto 3 Reparto 4 Reparto 5   
0       1  dicembre  2018    309.38       NaN       NaN       NaN       0.0   
1       2  dicembre  2018    551.91       NaN       NaN       NaN       0.0   
2       3  dicembre  2018    287.91       NaN       NaN       NaN       0.0   
3       4  dicembre  2018    416.46       NaN       NaN       NaN       0.0   
4       5  dicembre  2018    556.58       NaN       NaN       NaN       0.0   
5       6  dicembre  2018    320.02       NaN       NaN       NaN       0.0   
6       7  dicembre  2018    300.20       NaN       NaN       NaN       0.0   
7       8  dicembre  2018      0.00       NaN       NaN       NaN       0.0   
8       9  dicembre  2018      0.00       NaN       NaN       NaN       0.0   
9      10  dicembre  2018    347.69       NaN       NaN       NaN       0.0   
10     11  dicembre  2018    274.62       NaN       NaN       NaN       0.0   
11     12  dicembre  2018    361.90       NaN       NaN       NaN       0.0   
12     13  dicembre  2018    245.48       NaN       NaN       NaN       0.0   
13     14  dicembre  2018      0.00       NaN       NaN       NaN       0.0   
14     15  dicembre  2018    856.55       NaN       NaN       NaN       0.0   
15     16  dicembre  2018      0.00       NaN       NaN       NaN       0.0   
16     17  dicembre  2018    435.76       NaN       NaN       NaN       0.0   
17     18  dicembre  2018    212.82       NaN       NaN       NaN       0.0   
18     19  dicembre  2018    890.08       NaN       NaN       NaN       0.0   
19     20  dicembre  2018    257.62       NaN       NaN       NaN       0.0   
20     21  dicembre  2018    503.01       NaN       NaN       NaN       0.0   
21     22  dicembre  2018    238.30       NaN       NaN       NaN       0.0   
22     23  dicembre  2018      0.00       NaN       NaN       NaN       0.0   
23     24  dicembre  2018    466.28       NaN       NaN       NaN       0.0   
24     25  dicembre  2018      0.00       NaN       NaN       NaN       0.0   
25     26  dicembre  2018      0.00       NaN       NaN       NaN       0.0   
26     27  dicembre  2018    185.45       NaN       NaN       NaN       0.0   
27     28  dicembre  2018    206.09       NaN       NaN       NaN       0.0   
28     29  dicembre  2018    206.99       NaN       NaN       NaN       0.0   
29     30  dicembre  2018    386.69       NaN       NaN       NaN       0.0   
30     31  dicembre  2018      0.00       NaN       NaN       NaN       0.0   

     Cassa 2                                         Totali         \
   Reparto 1 Reparto 2 Reparto 3 Reparto 4 Reparto 5  IVA 4 IVA 10   
0       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
1       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
2       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
3       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
4      60.51       NaN       NaN       NaN       0.0    NaN    NaN   
5     196.46       NaN       NaN       NaN       0.0    NaN    NaN   
6       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
7     460.56       NaN       NaN       NaN       0.0    NaN    NaN   
8       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
9       0.00       NaN       NaN       NaN       0.0    NaN    NaN   
10      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
11      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
12    360.43       NaN       NaN       NaN       0.0    NaN    NaN   
13      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
14    530.66       NaN       NaN       NaN       0.0    NaN    NaN   
15      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
16      0.00       NaN       NaN       NaN       0.0    NaN    NaN   
17      0.00       NaN       NaN       NaN       0.0    NaN    NaN 